This code prints the fraction of emission in HII regions for the controls in the paper.


In [1]:
#import global packages
import numpy as np
import astropy.io.fits as pyfits
from astropy.wcs import WCS
import astropy.units as u
from astropy.table import Table
from reproject import reproject_interp
from astropy.coordinates import SkyCoord, Angle
from astropy.nddata import Cutout2D

# import local packages
import os, sys
sys.path.append('./Python')
from AngularSize import findAngSize

# map extensions
ext = "HA_FLUX_CORR"
hii_ext = 0


In [2]:
# Compile galaxy and sne data
data = Table.read('../Data/1.GroupedSampleData.csv', format='csv')

data[0:1]

col0,galaxy,distance,gal_ra,gal_dec,pos_ang,incl,nat_res,r_eff,maps_musedap,maps_nat,maps_150,maps_1kpc,maps_HII,maps_irac,maps_CO21,maps_eCO21,maps_ext,maps_vdisp,maps_HST,JWST200,JWST300,JWST335,JWST360,JWST770,JWST1000,JWST1130,JWST2100,sn_name,sn_type,sn_ra,sn_dec
int64,str7,float64,float64,float64,float64,float64,float64,float64,str71,str58,str64,str63,str50,str40,str72,str73,str36,str32,str45,str93,str93,str93,str93,str93,str94,str94,str94,str82,str50,str127,str132
0,NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,42.091877,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_150pc.fits,../../GalaxyFiles/MUSELargeFiles/SFR/copt/NGC1087_sfr_1kpc.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,../../GalaxyFiles/ALMACO2-1/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,../Data/SFR_v1/NGC1087_MAPS_sfr.fits,../Data/vdisp/ngc1087_vdisp.fits,../../GalaxyFiles/HST/ngc1087_hst_ha_sic.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F200W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F300W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F330W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F365W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F770W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1000W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F1130W_atGauss7p5_anchored.fits,../../GalaxyFiles/JWSTLargeFiles/Anchored_8.0_Gauss7.5/ngc1087_F2100W_atGauss7p5_anchored.fits,['SN1995V'],['II'],[41.61152777777777],[-0.4987861111111111]


In [3]:
# prepare lists for running control function

ccs = [(float(data["gal_ra"][i]), float(data["gal_dec"][i])) for i in range(len(data))] 
    
sn_ras, sn_decs, sn_names = [],[],[]
for i in range(len(data)):
    sn_ras.append(data["sn_ra"][i][1:-1].replace(",","").split(" "))
    sn_decs.append(data["sn_dec"][i][1:-1].replace(",","").split(" "))
    sn_names.append(data["sn_name"][i][1:-1].replace(",","").replace("'","").split(" "))

for i in range(len(sn_ras)):
    sn_ras[i] = [float(ele) for ele in sn_ras[i]]
    sn_decs[i] = [float(ele) for ele in sn_decs[i]]
    
sn_name = np.hstack(sn_names)

In [4]:
# returns the resolution used for the local control and the number of pixels in each control case
def randomLocal(localRes, ra, dec, galaxyDist, Ha_map, HII_map, sig_map, wcs):
    
    angSize = findAngSize(localRes, galaxyDist)

    #Get cutout fractions for controls
    #cutout2D needs skycoord position to carry units
    ra_SN, dec_SN = Angle(ra * u.degree), Angle(dec * u.degree)
    raRad, decRad  = ra_SN.radian * u.rad, dec_SN.radian * u.rad    
    position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
    size = u.Quantity((angSize,angSize), u.degree) #size is size of box in arcsec 
    # make 2D cutout, will assign a new wcs to cutout to keep track of coords
    HaCutout = Cutout2D(Ha_map, position, size, wcs) 
    # use this new wcs when converting to pixels to add additional details to plot
#         xval, yval = cutout.wcs.wcs_world2pix(ra,dec, 0)
    # get HII data for contours
    HIICutout = Cutout2D(HII_map, position, size, wcs)
    sigCutout = Cutout2D(CO_StN_repro, position, size, wcs)

    # Get pixels information for Control 1
    #flatten cutouts
    f_HII_cut = HIICutout.data.flatten()
    f_Ha_cut  = HaCutout.data.flatten()
    f_CO_cut  = sigCutout.data.flatten()

    #remove nans
    keep_cut = np.where(np.isfinite(f_Ha_cut))
    map_HII_cut = f_HII_cut[keep_cut]
    map_Ha_cut  = f_Ha_cut[keep_cut]
    map_CO_cut  = f_CO_cut[keep_cut]
    
    # find HII regions
    map_Ha_HII_cut = map_Ha_cut[np.where(map_HII_cut > 0)]               # Ha map WITH HII
    map_CO_HII_cut = map_CO_cut[np.where(map_HII_cut > 0)]               # CO map WITH HII
    map_Ha_HII_CO_cut = map_Ha_HII_cut[np.where(map_CO_HII_cut > 3.)]     # CO map WITH HII WITH CO
    
    # find non HII regions
    map_Ha_no_HII_cut = map_Ha_cut[np.where(map_HII_cut <= 0)]             # Ha map NO HII
    map_CO_no_HII_cut = map_CO_cut[np.where(map_HII_cut <= 0)]             # CO map NO HII
    map_Ha_no_HII_CO_cut = map_Ha_no_HII_cut[np.where(map_CO_no_HII_cut > 3.)] # CO map NO HII WITH CO

    # sum up the total pixels
    HaPix      = len(map_Ha_cut)                          # total number of map pixels
    HaHIIPix   = len(map_Ha_HII_cut)                      # pixels WITH HII
    HaHIICOPix = len(map_Ha_HII_CO_cut)                   # pixels WITH HII WITH CO
    HaCOPix    = len(map_Ha_no_HII_CO_cut)                # pixels NO HII WITH CO

    return(localRes, HaPix, HaHIICOPix, HaHIIPix, HaCOPix)


In [14]:
HaHIICO500,HaHIICO1000,HaHII500,HaHII1000,HaCO500,HaCO1000 = [],[],[],[],[],[]
HaHIICO_frac_500,HaCO_frac_500,HaHII_frac_500,HaHIICO_frac_1000,HaCO_frac_1000,HaHII_frac_1000 = [],[],[],[],[],[]


for i in range(len(data["galaxy"])):

    # Using MUSE map for coverage and to measure H-alpha emission
    Ha_hdu = pyfits.open(data["maps_nat"][i])
    Ha_map = Ha_hdu[ext].data  

    #Convert x & y pixels to ra and dec
    wcs      = WCS(Ha_hdu[ext].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
    
    #Get CO maps to look at the CO21 emission
    CO_hdu = pyfits.open(data["maps_CO21"][i])
    CO_map = CO_hdu[0].data
    
    #Get eCO maps to look at the CO21 emission error
    eCO_hdu = pyfits.open(data["maps_eCO21"][i])
    eCO_map = eCO_hdu[0].data  
    
    # clip so that we have no negatives or zeroes
    eCO_clip_map = np.nan_to_num(eCO_map, copy=True, nan=0.0, posinf=None, neginf=None)
    eCO_clip_map = np.clip(eCO_clip_map, 0.1, np.max(eCO_clip_map))
    
    # Make CO S/N maps
    CO_StN_map = CO_map/eCO_clip_map
    CO_StN_hdu = pyfits.PrimaryHDU(data=CO_StN_map, header=CO_hdu[0].header)
    
    #Get HII regions masks
    hdu_HII = pyfits.open(data["maps_HII"][i])
    HII_map  = hdu_HII[hii_ext].data        

    # Reproject CO StN maps
    CO_StN_repro, CO_foot = reproject_interp(CO_StN_hdu, Ha_hdu[ext].header)

    # find fractions of all control conditions
    for j in range(len(sn_ras[i])):
        
        #get number of pixels in each control case
        localRes500, HaPix500, HaHIICOPix500, HaHIIPix500, HaCOPix500 = randomLocal(500, sn_ras[i][j], sn_decs[i][j], data["distance"][i], Ha_map, HII_map, CO_StN_repro, wcs)
        localRes1000, HaPix1000, HaHIICOPix1000, HaHIIPix1000, HaCOPix1000 = randomLocal(1000, sn_ras[i][j], sn_decs[i][j], data["distance"][i], Ha_map, HII_map, CO_StN_repro, wcs)
        print("SN - Local, Ha Pixels, HII + CO, HII, CO ")
        print(j,localRes500, "     ", HaPix500, "     ", HaHIICOPix500, "  ", HaHIIPix500, " ", HaCOPix500)
        
        if(HaPix500!=0):
            HaHIICO_frac_500.append(HaHIICOPix500/float(HaPix500))
            HaHII_frac_500.append(HaHIIPix500/float(HaPix500))
            HaCO_frac_500.append(HaCOPix500/float(HaPix500))
        else:
            HaHIICO_frac_500.append(0)
            HaHII_frac_500.append(0)
            HaCO_frac_500.append(0)
            
        if(HaPix1000!=0):
            HaHIICO_frac_1000.append(HaHIICOPix1000/float(HaPix1000))
            HaHII_frac_1000.append(HaHIIPix1000/float(HaPix1000))
            HaCO_frac_1000.append(HaCOPix1000/float(HaPix1000))
        else:
            HaHIICO_frac_1000.append(0)
            HaHII_frac_1000.append(0)
            HaCO_frac_1000.append(0)
    
    print("On Galaxy: ", i+1, "of ", len(data["galaxy"]), data["galaxy"][i])    
    print("---------------------------------------")

HaCO5 = np.sum(HaCO_frac_500)/len(HaCO_frac_500)
HaHIICO5 = np.sum(HaHIICO_frac_500)/len(HaHIICO_frac_500)
HaHII5 = np.sum(HaHII_frac_500)/len(HaHII_frac_500)
HaCO10 = np.sum(HaCO_frac_1000)/len(HaCO_frac_1000)
HaHIICO10 = np.sum(HaHIICO_frac_1000)/len(HaHIICO_frac_1000)
HaHII10 = np.sum(HaHII_frac_1000)/len(HaHII_frac_1000)



SN - Local, Ha Pixels, HII + CO, HII, CO 
0 500       1089       308    394   361
On Galaxy:  1 of  10 NGC1087
---------------------------------------
SN - Local, Ha Pixels, HII + CO, HII, CO 
0 500       729       47    155   81
On Galaxy:  2 of  10 NGC1300
---------------------------------------
SN - Local, Ha Pixels, HII + CO, HII, CO 
0 500       676       0    0   0
SN - Local, Ha Pixels, HII + CO, HII, CO 
1 500       676       21    120   40
SN - Local, Ha Pixels, HII + CO, HII, CO 
2 500       676       114    114   478
SN - Local, Ha Pixels, HII + CO, HII, CO 
3 500       676       0    124   0
On Galaxy:  3 of  10 NGC1365
---------------------------------------
SN - Local, Ha Pixels, HII + CO, HII, CO 
0 500       784       51    125   24
On Galaxy:  4 of  10 NGC1433
---------------------------------------
SN - Local, Ha Pixels, HII + CO, HII, CO 
0 500       841       17    63   19
SN - Local, Ha Pixels, HII + CO, HII, CO 
1 500       841       119    129   603
SN - Local, H

In [15]:
print(np.sum(HaCO_frac_500)/len(HaCO_frac_500))
print(np.sum(HaHIICO_frac_500)/len(HaHIICO_frac_500))
print(np.sum(HaHII_frac_500)/len(HaHII_frac_500))

print(np.sum(HaCO_frac_1000)/len(HaCO_frac_1000))
print(np.sum(HaHIICO_frac_1000)/len(HaHIICO_frac_1000))
print(np.sum(HaHII_frac_1000)/len(HaHII_frac_1000))

0.2975808767933568
0.29186690872220494
0.3583296468371962
0.3287924331247962
0.2568678365312157
0.31887855237009893


In [16]:
# print control emission information
print("RANDOM LOCAL: Probability that a random pixel will be in HII, HII + CO, CO only for 500 and 1000pc")
print("------------")
print("Fraction of Local(500pc) Pixels in HII regions:",HaHII5)
print("Fraction of Local(500pc) Pixels in HII regions with CO:",HaHIICO5)
print("Fraction of Local(500pc) Pixels NOT in HII regions with CO:",HaCO5)
print("------------")
print("Fraction of Local(1000pc) Pixels in HII regions:",HaHII10)
print("Fraction of Local(1000pc) Pixels in HII regions with CO:",HaHIICO10)
print("Fraction of Local(1000pc) Pixels NOT in HII regions with CO:",HaCO10)

print()


RANDOM LOCAL: Probability that a random pixel will be in HII, HII + CO, CO only for 500 and 1000pc
------------
Fraction of Local(500pc) Pixels in HII regions: 0.3583296468371962
Fraction of Local(500pc) Pixels in HII regions with CO: 0.29186690872220494
Fraction of Local(500pc) Pixels NOT in HII regions with CO: 0.2975808767933568
------------
Fraction of Local(1000pc) Pixels in HII regions: 0.31887855237009893
Fraction of Local(1000pc) Pixels in HII regions with CO: 0.2568678365312157
Fraction of Local(1000pc) Pixels NOT in HII regions with CO: 0.3287924331247962



In [ ]:
HaTot, HaHII, SDTot, SDHII, COHII, COTot = [],[],[],[],[],[]
HaPix, HaHIIPix, HaHIICOPix, HaCOPix = [],[],[],[]
SDPix, SDHIIPix, SDHIICOPix, SDCOPix = [],[],[],[]
COPix, COHIIPix = [],[]
HaHIICO, HaCO, SDCO, SDHIICO = [],[],[],[]
HaHIICO500 = []
HaHIICO1000 = []
HaHII500 = []
HaHII1000 = []
HaCO500 = []
HaCO1000 = []
HaHIICO_frac_500,HaCO_frac_500,HaHII_frac_500,HaHIICO_frac_1000,HaCO_frac_1000,HaHII_frac_1000 = [],[],[],[],[],[]


for i in range(len(data["galaxy"])):
# for i in range(0,2):    
    # Using MUSE map for coverage and to measure H-alpha emission
    Ha_hdu = pyfits.open(data["maps_nat"][i])
    Ha_map = Ha_hdu[ext].data  

    #Convert x & y pixels to ra and dec
    wcs      = WCS(Ha_hdu[ext].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
            
    #flatten data structures 
    f_Ha     = Ha_map.flatten()    
    f_ra     = ra.flatten()
    f_dec    = dec.flatten()    
    
    #Get IRAC maps to look at the stellar populations emission
    SD_hdu = pyfits.open(data["maps_irac"][i])
    SD_map = SD_hdu[0].data
    
    #Get CO maps to look at the CO21 emission
    CO_hdu = pyfits.open(data["maps_CO21"][i])
    CO_map = CO_hdu[0].data
    
    #Get eCO maps to look at the CO21 emission error
    eCO_hdu = pyfits.open(data["maps_eCO21"][i])
    eCO_map = eCO_hdu[0].data  
    # clip so that we have no negatives or zeroes
    eCO_clip_map = np.nan_to_num(eCO_map, copy=True, nan=0.0, posinf=None, neginf=None)
    eCO_clip_map = np.clip(eCO_clip_map, 0.1, np.max(eCO_clip_map))
    
    # Make CO S/N maps
    CO_StN_map = CO_map/eCO_clip_map

    CO_StN_hdu = pyfits.PrimaryHDU(data=CO_StN_map, header=CO_hdu[0].header)
    
    #Get HII regions masks
    hdu_HII = pyfits.open(data["maps_HII"][i])
    HII_map  = hdu_HII[hii_ext].data        
    
    # Reproject IRAC maps
    SD_repro, SD_foot = reproject_interp(SD_hdu, Ha_hdu[ext].header)

    # Reproject CO maps
    CO_repro, CO_foot = reproject_interp(CO_hdu, Ha_hdu[ext].header)
    
    # Reproject CO StN maps
    CO_StN_repro, CO_foot = reproject_interp(CO_StN_hdu, Ha_hdu[ext].header)

    #Convert x & y pixels to ra and dec
    wcs      = WCS(hdu_HII[0].header, naxis=2)
    naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid     = np.indices((naxis[1],naxis[0]))
    ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)
            
    #flatten data structures 
    f_HII    = HII_map.flatten()
    f_ra     = ra.flatten()
    f_dec    = dec.flatten()    
    f_Ha     = Ha_map.flatten()
    f_SD     = SD_repro.flatten() 
    f_CO     = CO_repro.flatten()
    f_CO_StN = CO_StN_repro.flatten()
    
    #remove nans for Halpha only
    keep     = np.where(np.isfinite(f_Ha) & np.isfinite(f_HII))
    map_HII  = f_HII[keep]
    map_Ha   = f_Ha[keep] 
    
    #remove nans for Halpha and IRAC overlap
    keep_SD    = np.where(np.isfinite(f_Ha) & np.isfinite(f_HII) & np.isfinite(f_SD))
    map_Ha_SD  = f_Ha[keep_SD] 
    map_SD     = f_SD[keep_SD]
    map_HII_SD = f_HII[keep_SD]

    #remove nans for Halpha and CO overlap
    keep_CO     = np.where(np.isfinite(f_Ha) & np.isfinite(f_HII) & np.isfinite(f_CO_StN))
    map_Ha_CO   = f_Ha[keep_CO] 
    map_CO      = f_CO_StN[keep_CO]
    map_HII_CO  = f_HII[keep_CO]

    # count number of pixels in each
    HaPix.append(len(map_Ha))
    SDPix.append(len(map_SD))
    COPix.append(len(map_CO))
    
    # sum up the total emission
    HaTot.append(np.sum(map_Ha)) 
    SDTot.append(np.sum(map_SD))
    COTot.append(np.sum(map_CO))
    
    #Isolate HII regions from SD maps    
    map_SD_HII    = map_SD[np.where(map_HII_SD >= 0)]
    map_SD_no_HII = map_SD[np.where(map_HII_SD < 0)]

    #Isolate HII regions from CO maps
    map_CO_HII     = map_CO[np.where(map_HII_CO >= 0)] # CO map in HII regions
    map_CO_no_HII  = map_CO[np.where(map_HII_CO < 0)]  # CO map NOT in HII regions
        
    # Halpha, CO, and HII 
    map_Ha_HII       = map_Ha_CO[np.where(map_HII_CO >= 0)]        # Ha in HII regions (CO map coverage)
    map_Ha_no_HII    = map_Ha_CO[np.where(map_HII_CO < 0)]         # Ha NOT in HII regions (CO map coverage)
    map_Ha_HII_CO    = map_Ha_HII[np.where(map_CO_HII > 3.)]       # Ha emission in HII regions with CO Signal
    map_Ha_no_HII_CO = map_Ha_no_HII[np.where(map_CO_no_HII > 3.)] # Ha emission NOT in HII regions with CO Signal
    map_SD_HII_CO    = map_SD_HII[np.where(map_CO_HII > 3.)]  
    map_SD_no_HII_CO = map_SD_no_HII[np.where(map_CO_no_HII > 3.)]  
    
    # gather each galaxies number of pixels meeting condition
    HaHIIPix.append(len(map_Ha[np.where(map_HII>=0)]))       # H-alpha pixels in HII regions
    SDHIIPix.append(len(map_SD_HII))                         # IRAC pixels in HII regions
    COHIIPix.append(len(map_CO_HII))                         # CO pixels in HII regions
    HaHIICOPix.append(len(map_Ha_HII_CO))                    # CO pixels in HII regions WITH Ha
    HaCOPix.append(len(map_Ha_no_HII_CO))                    # CO pixels NOT in HII regions WITH Ha

    # gather each galaxies sum of the HII region emission
    HaHII.append(np.sum(map_Ha_HII))
    SDHII.append(np.sum(map_SD_HII))
    COHII.append(np.sum(map_CO_HII)) 
    HaHIICO.append(np.sum(map_Ha_HII_CO))
    HaCO.append(np.sum(map_Ha_no_HII_CO))
    SDHIICO.append(np.sum(map_SD_HII_CO))
    SDCO.append(np.sum(map_SD_no_HII_CO))

    
    print("On Galaxy: ", i+1, "of ", len(data["galaxy"]), data["galaxy"][i])    
    print("---------------------------------------")
        
    

In [ ]:
# print control emission information
print("EMISSION")
print("--------")
print("Fraction of Ha Emission in HII regions",np.sum(HaHII)/np.sum(HaTot))
print()
print("Fraction of SD Emission in HII regions",np.sum(SDHII)/np.sum(SDTot))
print()
print("Fraction of CO Emission in HII regions",np.sum(COHII)/np.sum(COTot))
print("--------")
print()

print("EMISSION WITH/OUT CO HII")
print("------------------------")

print("Fraction of Ha emission in HII regions with coincident CO21 emission",
      np.sum(HaHIICO)/np.sum(HaTot))
print("Fraction of Ha emission NOT in HII regions with coincident CO21 emission",
      np.sum(HaCO)/np.sum(HaTot))
print()
print("Fraction of IRAC emission in HII regions with coincident CO21 emission",
      np.sum(SDHIICO)/np.sum(SDTot))
print("Fraction of IRAC emission NOT in HII regions with coincident CO21 emission",
      np.sum(SDCO)/np.sum(SDTot))
print()
print("-----------------")

print("RANDOM MAP PIXELS")
print("-----------------")
print("Probability that a random pixel will be in HII, HII + CO, CO only")
print("Fraction of Pixels in HII regions",np.sum(HaHIIPix)/np.sum(HaPix))
print("Fraction of Pixels in HII regions with CO",np.sum(HaHIICOPix)/np.sum(HaPix))
print("Fraction of Pixels NOT in HII regions with CO",np.sum(HaCOPix)/np.sum(HaPix))

# print("Fraction of Ha Pixels in HII regions",np.sum(HaHIIPix)/np.sum(HaPix))
# print("Fraction of SD Pixels in HII regions",np.sum(SDHIIPix)/np.sum(SDPix))
# print("Fraction of CO Pixels in HII regions",np.sum(COHIIPix)/np.sum(COPix))

print()
